In [ ]:
#@title Mouting Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Imports

In [ ]:
#@title Import Libraries
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import wave

In [ ]:
#@title Unzip Files
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Crema.zip", 'r')
zip_ref.extractall("/content/drive/MyDrive/Audio Files ")
zip_ref.close()

# Process Audios

In [ ]:
from types import AsyncGeneratorType
import os

path = "/content/drive/MyDrive/Machine Learning/SER/Audio Files /"

sadness=[]
angry=[]
disgust=[]
fear=[]
happy=[]
neutral=[]
labels=[]
for filename in os.listdir(path):
    if "SAD" in filename:
        sadness.append(filename)
        labels.append("Sad")

    elif "ANG" in filename:
        angry.append(filename)
        labels.append("Angry")
    elif "DIS" in filename:
        disgust.append(filename)
        labels.append("Disgust")
    elif "FEA" in filename:
        fear.append(filename)
        labels.append("Fear")
    elif "HAP" in filename:
        happy.append(filename)
        labels.append("Happy")

    elif "NEU" in filename:
        neutral.append(filename)
        labels.append("Neutral")

In [ ]:
print("sadnes size",len(sadness))
print("angry size",len(angry))
print("disgust size",len(disgust))
print("fear size",len(fear))
print("happy size",len(happy))
print("neutral size",len(neutral))
print("labels size",len(labels))


sadnes size 1271
angry size 1271
disgust size 1271
fear size 1271
happy size 1271
neutral size 1087
labels size 7442


In [ ]:
#@title Reading Audio Using Librosa Library
import librosa
import librosa.display
import matplotlib.pyplot as plt

def load(path):
  audios=[]
  for filename in os.listdir(path): 
    y, sr = librosa.load(f'/content/drive/MyDrive/Machine Learning/SER/Audio Files /{filename}')
    audios.append(y)
  return audios,sr  


  # plt.figure(figsize=(15,5))
  # plt.plot(y);
  # plt.title('Signal');
  # plt.xlabel('Time (samples)');
  # plt.ylabel('Amplitude');

In [ ]:
#@title Shift Pitch of Waveform
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pitch_shifting(audios,sr):
  pitch_shifted=[]
  for i in range (len(audios)):
      pitch_shifted.append(librosa.effects.pitch_shift(audios[i], sr=sr, n_steps=4))
  return pitch_shifted


#Time Stretching of Waveform
def time_stretching(audios):
  time_stretched=[]
  for i in range (len(audios)):
      time_stretched.append(librosa.effects.time_stretch(audios[i], rate=0.8))
  return time_stretched






In [ ]:

# Zero Crossing Rate
def zcr(data):
    frame_length=2048
    hop_length=512
    zcr = librosa.feature.zero_crossing_rate(y=data, frame_length=frame_length, hop_length=hop_length)
    return np.squeeze(zcr)


def energy(data):
    frame_length=2048
    hop_length=512
    en = np.array([np.sum(np.power(np.abs(data[hop:hop+frame_length]), 2)) for hop in range(0, data.shape[0], hop_length)])
    return en / frame_length



In [ ]:
# def concat(audio, time_stretched, pitch_shifted):
#     # Concatenate the data
#     concatenated_data = np.concatenate((audio, time_stretched, pitch_shifted), axis=0)
#     return concatenated_data

def padding(audios):
  max_size=max(map(len,audios))
  padded_audios = [np.pad(audio, (0, max_size - len(audio)), mode='constant') for audio in audios]

  return padded_audios

#       # sequence_list = [list(audio) for audio in audios]
#       # print(type(sequence_list))
#       # print(type(sequence_list[0]))
#       # padded_data = pad_sequences(sequence_list, padding='post')



In [ ]:
audios,sr = load( "/content/drive/MyDrive/Machine Learning/SER/Audio Files /")

In [ ]:
sound=[]
for audio in audios:
  data=np.array([])
  data=np.hstack((data,zcr(audio),energy(audio)))
  sound.append(data)


In [ ]:
padded_sounds=padding(sound)

In [ ]:
audios = np.vstack(padded_sounds)

In [ ]:
# audiosPad = padding(audios)
# audios = np.vstack(audiosPad)

In [ ]:
# #@title Load Audios Concat Audios when session Crash
# audios = np.load("/content/drive/MyDrive/Machine Learning/SER/Concat_Audios.npy",allow_pickle=True)
# #@title Tripling Output 
# labels=labels+labels+labels

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Convert string labels to numeric labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [ ]:
len(audios)

7442

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(audios,labels, test_size=0.3, random_state=42, stratify=labels)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train, test_size=0.05, random_state=42, stratify=y_train)
# Check out the data
print(f'X_train shape: {len(x_train)}')
print(f'y_train shape: {len(y_train)}')
print(f'X_test shape: {len(x_test)}')
print(f'y_test shape: {len(y_test)}')

X_train shape: 4948
y_train shape: 4948
X_test shape: 2233
y_test shape: 2233


In [ ]:
#@title VGG
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense,Dropout

# Create a Sequential model
model = Sequential()

# Add 1D convolutional layers
model.add(Conv1D(64, 3, activation='relu', padding='same', input_shape=(len(x_train[0]), 1)))
model.add(Conv1D(64, 3, activation='relu', padding='same'))

model.add(MaxPooling1D(2, strides=2))

model.add(Conv1D(128, 3, activation='relu', padding='same'))
model.add(Conv1D(128, 3, activation='relu', padding='same'))

model.add(MaxPooling1D(2, strides=2))

model.add(Conv1D(256, 3, activation='relu', padding='same'))
model.add(Conv1D(256, 3, activation='relu', padding='same'))
model.add(Conv1D(256, 3, activation='relu', padding='same'))

model.add(MaxPooling1D(2, strides=2))

model.add(Conv1D(512, 3, activation='relu', padding='same'))
model.add(Conv1D(512, 3, activation='relu', padding='same'))
model.add(Conv1D(512, 3, activation='relu', padding='same'))

model.add(MaxPooling1D(2, strides=2))

model.add(Conv1D(512, 3, activation='relu', padding='same'))
model.add(Conv1D(512, 3, activation='relu', padding='same'))
model.add(Conv1D(512, 3, activation='relu', padding='same'))

model.add(MaxPooling1D(2, strides=2))

# Flatten the feature maps
model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=128, activation='relu'))
model.add(Dense(6, activation='softmax'))

model.summary()


In [ ]:
#@title Simple Architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

model = Sequential()

# Convolutional layers
model.add(Conv1D(64, 3, activation='relu', padding='same', input_shape=(len(x_train[0]), 1)))

model.add(MaxPooling1D(2, strides=2))

model.add(Conv1D(128, 3, activation='relu', padding='same'))

model.add(MaxPooling1D(2, strides=2))

model.add(Conv1D(256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(512, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
# Flatten the output for the fully connected layers
model.add(Flatten())

# Fully connected layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))  # num_classes is the number of output classes

# Print the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 432, 64)           256       
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 216, 64)          0         
 1D)                                                             
                                                                 
 conv1d_9 (Conv1D)           (None, 216, 128)          24704     
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 108, 128)         0         
 1D)                                                             
                                                                 
 conv1d_10 (Conv1D)          (None, 106, 256)          98560     
                                                                 
 max_pooling1d_10 (MaxPoolin  (None, 53, 256)         

In [ ]:
# Compile the model
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# 5. Train the model 

model.fit(x_train, y_train, batch_size=16, epochs=50, validation_data=(x_val, y_val))


Epoch 1/50
310/310 [==============================] - 2s 5ms/step - loss: 1.7913 - accuracy: 0.1645 - val_loss: 1.7649 - val_accuracy: 0.2490
Epoch 2/50
310/310 [==============================] - 1s 4ms/step - loss: 1.7301 - accuracy: 0.2603 - val_loss: 1.6698 - val_accuracy: 0.2797
Epoch 3/50
310/310 [==============================] - 1s 4ms/step - loss: 1.6429 - accuracy: 0.3153 - val_loss: 1.6108 - val_accuracy: 0.3487
Epoch 4/50
310/310 [==============================] - 1s 4ms/step - loss: 1.6008 - accuracy: 0.3323 - val_loss: 1.6327 - val_accuracy: 0.3333
Epoch 5/50
310/310 [==============================] - 1s 4ms/step - loss: 1.5825 - accuracy: 0.3438 - val_loss: 1.5603 - val_accuracy: 0.3333
Epoch 6/50
310/310 [==============================] - 1s 4ms/step - loss: 1.5791 - accuracy: 0.3446 - val_loss: 1.5469 - val_accuracy: 0.3448
Epoch 7/50
310/310 [==============================] - 1s 4ms/step - loss: 1.5698 - accuracy: 0.3543 - val_loss: 1.5568 - val_accuracy: 0.3678
Epoch 

In [ ]:
# Train the model
# model.fit_ge(X_train,y_train,epochs=10 ,batch_size=128)
# eval the model
loss,accuracy = model.evaluate(x_test,y_test)

# Make predictions with the trained model
predictions = model.predict(x_test)

In [ ]:
#@title dont do this at home
# Import necessary libraries
import librosa
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
from ipywidgets import interactive
import soundfile as sf
import librosa.display
from IPython.display import Audio
# sf.write('input.wav', y, 44100)
# Load the input recording
data, sr = librosa.load('/content/drive/MyDrive/Audio Files /1001_DFA_HAP_XX.wav', sr=44100)

# Display the input audio file in time domain
plt.figure(figsize=(15, 5))
plt.plot(data)
plt.title('Input Audio Waveform')
plt.xlabel('Time (s)')
plt.show()

# Set frame length, hop length, and ZCR threshold
framelength = 2000
hoplength = int(framelength/4)
zcr_threshold = 0.1

# Compute ZCR of the audio signal using librosa
zcr = librosa.feature.zero_crossing_rate(data, frame_length=framelength, hop_length=hoplength)[0]

# Plot the computed ZCR
plt.figure(figsize=(15, 5))
plt.plot(zcr)
plt.axhline(y=zcr_threshold, color='r', linestyle='-')
plt.title('Zero-Crossing Rate')
plt.xlabel('Frame Index')
plt.ylabel('ZCR')
plt.show()

# Create 'data' array containing the input file
data = np.array(data)

# Set frame start and end positions to 0
framestart = 0
frameend = 0
count = 0
count2=0
# Iterate through 'zcr' array with 'index' as the iterator
for index in range(len(zcr)):
    # If 'zcr' value at 'index' is less than the threshold, increment frame start and end positions by hop length
    
    if zcr[index] < zcr_threshold:
        count+=1
        framestart += hoplength
        frameend += hoplength
        
    # If frame end position exceeds length of 'data' array, break loop
    elif frameend + hoplength > len(data):
        break
    # Otherwise, set values in 'data' array to 2 within the current frame
    else:
        count2+=1
        for i in range(framestart, frameend - hoplength):
            print("LOLO")
            data[i] = 2
        # Move on to next frame
        framestart += hoplength
        frameend += hoplength
    
# Iterate through 'data' array and list the values that are not 2 in 'output' array
print(count)
print(count2)
output = []

for i in range(len(data)):
    if data[i] != 2:
        output.append(data[i])
output = np.array(output)

# Plot the 'output' array, representing the output waveform with the noise removed
plt.figure(figsize=(15, 5))
plt.plot(output)
plt.title('Output Audio Waveform')
plt.xlabel('Time (samples)')
# Save the output audio file

sf.write('output.wav', output, sr)


print(len(data))
print(len(output))

# Play both input and output audio files using widgets
input_widget = Audio(data, rate=sr)
output_widget = Audio(output, rate=sr)

def play_audio(input_audio, output_audio):
    display(input_audio)
    display(output_audio)

play_audio(input_widget,output_widget)


In [ ]:
#@title dont do this at home
# compute STAZCR

stazcr = librosa.feature.zero_crossing_rate(y)

# set thresholds
voiced_threshold = 0.05
unvoiced_threshold = 0.2

# segment signal based on thresholds
segments = []
start = 0
for i in range(len(stazcr[0])):
    if stazcr[0][i] < voiced_threshold:
        if start == 0:
            start = i
    elif stazcr[0][i] > unvoiced_threshold:
        if start != 0:
            segments.append((start, i))
            start = 0
if start != 0:
    segments.append((start, len(stazcr[0])))

# extract voiced and unvoiced segments
voiced_segments = []
unvoiced_segments = []
for segment in segments:
    if stazcr[0][segment[0]] < voiced_threshold:
        # voiced segment
        voiced_segments.append(y[segment[0]:segment[1]])
    else:
        # unvoiced segment
        unvoiced_segments.append(y[segment[0]:segment[1]])

# print number of voiced and unvoiced segments
print('Number of voiced segments:', len(voiced_segments))
print('Number of unvoiced segments:', len(unvoiced_segments))

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Load audio file
# y, sr = librosa.load('/content/1001_DFA_ANG_XX.wav')

# Compute mel spectrogram
n_fft = 2048
hop_length = 512
n_mels = 128
fmin = 20
fmax = sr // 2
S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, fmin=fmin, fmax=fmax)

# Convert to decibels
S_dB = librosa.power_to_db(S, ref=np.max)

# Plot spectrogram
plt.figure(figsize=(10, 4))
librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=sr, fmin=fmin, fmax=fmax)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-frequency spectrogram')
plt.tight_layout()
plt.show()


In [ ]:
#Hear Sound
from IPython.display import Audio
wn = Audio('/content/drive/MyDrive/Audio Files /1007_IWW_ANG_XX.wav', autoplay=True)
display(wn)